Assamese -fastetxt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


preprocesing

In [ ]:
import pandas as pd

In [ ]:
!pip install emot --upgrade
# to convert emojis to text
!pip install emoji
#to expand a contracted words
!pip install demoji
!pip install contractions

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.naive_bayes import MultinomialNB
from textblob import TextBlob
import emot
import nltk.data
import numpy as np
import pandas as pd
import re
import time
import string
import emoji
import demoji
import contractions
import nltk
nltk.download('omw-1.4')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option("max_colwidth" ,220)
wn = nltk.WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

english_stopwords = stopwords.words("english")
with open('/content/drive/MyDrive/hasoc/task4/assam/assamese.txt', encoding = 'utf-8') as f:
    assam_stopwords = f.readlines()
    for i in range(len(assam_stopwords)):
        assam_stopwords[i] = re.sub('\n','', assam_stopwords[i])
stopwords = english_stopwords + assam_stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
len(stopwords)

295

In [ ]:
demoji.download_codes()
def emo(text):
  temp=emoji.demojize(text,delimiters=(" "," "))
  temp=temp.replace("_","  ")
  return temp

<ipython-input-47-e00aa54160fe>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
demoji.download_codes()
def emo(text):
  try:
   temp=emoji.demojize(str(text),delimiters=(" "," "))
   temp=temp.replace("_","  ")
   return temp
  except IndexError:
        return ""

<ipython-input-48-82a5fb4117d1>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
punct="!#$%&\'()*+,-.’’/:;<=>?@[\\]^_`{|}~’“‘"

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', str(text))

In [ ]:
def clean_text(text):
    text = re.sub('@[a-zA-Z0-9]*', '', str(text))#to remove @ and its following word
    text = contractions.fix(text, slang=True)
    text = "".join([word.lower() for word in text if word not in string.punctuation]) #to remove punctuation
    text="".join([word.lower() for word in text if word not in punct])
    text = "".join([word for word in text if not word.isdigit()])#to remove digit
    text = " ".join(word for word in text.split() if word not in stopwords)
    if len(text) == 0:
      text= '**********************'
    return text

In [ ]:
asa_train=pd.read_csv('/content/drive/MyDrive/hasoc/task4/assam/train_A_AH_HASOC2023.csv')

In [ ]:
asa_train['task_1'].value_counts()

HOF    2347
NOT    1689
Name: task_1, dtype: int64

In [ ]:
asa_train['clean_text'] = asa_train['text'].apply(lambda x:emo(x))
asa_train['clean_text'] = asa_train['clean_text'].apply(lambda x:remove_urls(x))
asa_train['clean_text'] = asa_train['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
asa_train[asa_train['clean_text'] =='**********************']

,S. No.,text,task_1,clean_text
2107,2108,হা হা হা হা হা হা হা হা হা,NOT,**********************


In [ ]:
asa_train['clean_text']

0                                                                                                                                      লাগিলে ইযাৰ পৰাই কলি লাগাম
1                                                                         আমি গেদা মই গেদা rolling floor laughing rolling floor laughing rolling floor laughing ।
2                                      মানুহে যেতিয়া নিজৰ জীৱন টো নিজৰ কাৰণে সমাজৰ কাৰণে দেশৰ কাৰণে পৃথিৱীৰ কাৰণে কিমান মূল্যবান বুজি নাপায় তেতিয়া এনেকুৱা হয়
3                                                                                                                   কতি খন কোন দিনা দেখা পাও তাকে বাত চায় আছো মই
4       কাপোৰ খন চাইকেল অহা মানুহজনৰ মুখত দলিয়াব লাগেনে বেছেৰাটো গুচি গল সেইকাৰণে বেয়া পায়। rolling floor laughing rolling floor laughing rolling floor laughing
                                                                                  ...                                                                            
4031                        

In [ ]:
asa_train.shape

(4036, 4)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(asa_train.clean_text, asa_train.task_1, test_size=0.25, random_state=42)

In [ ]:
X_train.shape, X_test.shape

((3027,), (1009,))

In [ ]:
!pip install fasttext

  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199773 sha256=3dd6a71ef844e3af13f17f015cd44c8995b3d9292d42ce19c731d95511fb5428
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import gzip
import numpy as np

In [ ]:
embedding_index_asa = {}
asavocab = []


file_path = '/content/drive/MyDrive/hasoc/task4/assam/cc.as.300.vec.gz'

# Open the gzipped file
with gzip.open(file_path, 'rt', encoding='utf-8') as f:
    for line in f:
        values = line.rstrip().split(' ')
        word = values[0]
        asavocab.append(word)
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index_asa[word] = coefs

In [ ]:
embedding_index_eng = {}
engvocab = []


file_path = '/content/drive/MyDrive/hasoc/task4/assam/cc.en.300 (1).vec.gz'

# Open the gzipped file
with gzip.open(file_path, 'rt', encoding='utf-8') as f:
    for line in f:
        values = line.rstrip().split(' ')
        word = values[0]
        engvocab.append(word)
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index_eng[word] = coefs

In [ ]:
len(asavocab)

295259

In [ ]:
len(engvocab)

2000001

In [ ]:
def get_vectors(sentence):
    sentvec = np.zeros(300)
    zeros = np.zeros(300)

    for word in sentence.split():
        if word in engvocab:
            temp = embedding_index_eng[word]
        elif word in asavocab:
            temp = embedding_index_asa[word]
        else:
            temp = zeros
        # print('word : ', word, ' ttt : ', temp)
        sentvec += temp

    sentvec = sentvec / len(sentence.split())
    return sentvec

In [ ]:
X_train['vectors'] = asa_train['clean_text'].apply(lambda x:get_vectors(x))
X_test['vectors'] = asa_train['clean_text'].apply(lambda x:get_vectors(x))

In [ ]:
type(X_train['vectors'] )

pandas.core.series.Series

In [ ]:
zeros=([0.]*300)
zeros=np.array(zeros)

In [ ]:
asa_train['clean_text'][2107]=zeros

<ipython-input-76-a95c6568a06b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asa_train['clean_text'][2107]=zeros


In [ ]:
X_train=X_train['vectors'].to_list()
X_test=X_test['vectors'].to_list()

In [ ]:
y_train =asa_train['task_1'].to_list()
y_test=asa_train['task_1'].to_list()

In [ ]:
type(y_train)

list

SVM

In [ ]:
from sklearn.svm import SVC
svm = SVC()

svm .fit(X_train, y_train)

y_pred = svm.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", round(accuracy*100, 4))

print("\n", classification_report(y_test, y_pred))

Ensemble classifier

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC

clf1 = LogisticRegression(random_state=1)
clf2 = BernoulliNB()
clf3 = LinearSVC(penalty='l2',C=1.0)

eclf = VotingClassifier(estimators=[('LR', clf1), ('BNB', clf2),
('SVC', clf3)], voting='hard')
eclf = eclf.fit(X_train, y_train)
y_pred_df= eclf.predict(X_test)
print(y_pred_df)
print("Accuracy of %s is %s"%(eclf, acc))
print("\n", classification_report(y_test, y_pred_df))